### Import Libraries

In [1]:
import numpy as np
import pandas as pd


### Load and explore the data (4 marks)

In [2]:
df = pd.read_csv('product-cat-dataset.csv')

In [3]:
#View the first lines of dataset to check the contents

df.head()

,Description,Level_1,Level_2,Level_3
0,gerb cap help keep littl on head cov warm day ...,09BF5150,C7E19,D06E
1,newborn inf toddl boy hoody jacket oshkosh b g...,2CEC27F1,ADAD6,98CF
2,tut ballet anym leap foxy fash ruffl tul toddl...,09BF5150,C7E19,D06E
3,newborn inf toddl boy hoody jacket oshkosh b g...,2CEC27F1,ADAD6,98CF
4,easy keep feel warm cozy inf toddl girl hoody ...,2CEC27F1,ADAD6,98CF


In [4]:
#Obtain information about the different columns in the dataset including

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10649 entries, 0 to 10648
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Description  10637 non-null  object
 1   Level_1      10649 non-null  object
 2   Level_2      10649 non-null  object
 3   Level_3      10649 non-null  object
dtypes: object(4)
memory usage: 332.9+ KB


In [5]:
df.describe()

,Description,Level_1,Level_2,Level_3
count,10637,10649,10649,10649
unique,9677,15,39,43
top,glory gorg col fing complet outfit express moo...,B092BA29,2D5A3,28A7
freq,24,900,797,797


In [6]:
df.agg(['count', 'size', 'nunique'])

,Description,Level_1,Level_2,Level_3
count,10637,10649,10649,10649
size,10649,10649,10649,10649
nunique,9677,15,39,43


### Deal with Missing Data (4 marks)

In [7]:
# Check if data has missing values in the Description column
#Check column distribution of null values and their proportions

Column_Info= pd.DataFrame(df.dtypes).T.rename(index={0:'column type'})
Column_Info= Column_Info.append(pd.DataFrame(df.isnull().sum()).T.rename(index={0:'null values (nb)'}))
Column_Info= Column_Info.append(pd.DataFrame(df.isnull().sum()/df.shape[0]*100).T.
                                rename(index={0:'null values (%)'}))
display(Column_Info)

,Description,Level_1,Level_2,Level_3
column type,object,object,object,object
null values (nb),12,0,0,0
null values (%),0.112687,0.0,0.0,0.0


In [8]:
# Deal with missing values
#Remove Null Values
df.dropna(axis = 0, subset = ['Description'], inplace = True)

#Check null values in dataset and technical info on the columns
df.isnull().sum()


Description    0
Level_1        0
Level_2        0
Level_3        0
dtype: int64

### Drop Classes where the number of instances is < 10 (4 marks)

In [9]:
def delete_instances(data, level):
    level_data = data[[level, 'Description']]
    level_grouped =  level_data.groupby([level]).size().reset_index(name='counts')
    instances_to_delete = level_grouped[level_grouped['counts'] < 10][level].tolist()
    print(instances_to_delete)
    data.drop(data[data[level].isin(instances_to_delete)].index, inplace=True)
    return data


In [10]:
# Apply to Level_1
df = delete_instances(df, 'Level_1')

[]


In [11]:
# Apply to Level_2
df = delete_instances(df, 'Level_2')

['80D5B', 'A6301', 'C66C5']


In [12]:
# Apply to Level_3
df = delete_instances(df, 'Level_3')


['CF52', 'DE3D']


### Now let's write a Function to Prepare Text (4 marks)
We will apply it to our DataFrame later on

* This function receives a text string and performs the following:
* Convert text to lower case
* Remove punctuation marks
* Apply stemming using the popular Snowball or Porter Stemmer (optional)
* Apply NGram Tokenisation
* Return the tokenised text as a list of strings

In [36]:
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import PorterStemmer
from nltk import word_tokenize
from nltk.util import ngrams
import string

#Download packages for nltk
#nltk.download()


def process_text(text, n = 1):
    """
    Takes in a string of text, then performs the following:
    1. Convert text to lower case and remove all punctuation
    2. Optionally apply stemming
    3. Apply Ngram Tokenisation
    4. Returns the tokenised text as a list
    """
    # write steps here

    #Explicitly calling lower here to comply with instructions, however lower is being called anyway by PorterStemmer
    text = text.lower()
    #Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    #Apply tokenisation and stemming
    tokenised = [ ' '.join(grams) for grams in ngrams(word_tokenize(text), n)]
    
    return tokenised

'''
Same function but with stemming applied.
Please note that the output of this function is a bit different though.
'''
def process_text_stem(text, n = 1):
    """
    Takes in a string of text, then performs the following:
    1. Convert text to lower case and remove all punctuation
    2. Optionally apply stemming
    3. Apply Ngram Tokenisation
    4. Returns the tokenised text as a list
    """
    # write steps here

    #Explicitly calling lower here to comply with instructions, however lower is being called anyway by PorterStemmer
    text = text.lower()
    #Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    ps = PorterStemmer()
    #Apply tokenisation and stemming
    tokenised = [ ps.stem(' '.join(grams)) for grams in ngrams(word_tokenize(text), n)]

    return tokenised




In [37]:
# Here is an example function call
process_text("Here we're testing the process_text function, results are as follows:", n = 3)

['here were testing',
 'were testing the',
 'testing the processtext',
 'the processtext function',
 'processtext function results',
 'function results are',
 'results are as',
 'are as follows']

In [38]:
#Execution of the same function but with stemming applied. The output is slightly different to the example provided.
process_text_stem("Here we're testing the process_text function, results are as follows:", n = 3)

['here were test',
 'were testing th',
 'testing the processtext',
 'the processtext funct',
 'processtext function result',
 'function results ar',
 'results are a',
 'are as follow']

In [39]:
# Results should look like this:
['here were test',
 'were test the',
 'test the processtext',
 'the processtext function',
 'processtext function result',
 'function result are',
 'result are as',
 'are as follow']

['here were test',
 'were test the',
 'test the processtext',
 'the processtext function',
 'processtext function result',
 'function result are',
 'result are as',
 'are as follow']

### Now let's apply TF-IDF to extract features from plain text (10 marks)

In [5]:
# Might take a while...
# Here you apply the process_text function to the Description column of the data
# Then you pass the results to the bag of words tranformer
# See here: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html



Now we can use .transform on our Bag-of-Words (bow) transformed object and transform the entire DataFrame of text file contents. Let's go ahead and check out how the bag-of-words counts for the entire corpus in a large, sparse matrix:

In [15]:
# After that you pass the result of the previous step to sklearn's TfidfTransformer
# which will convert them into a feature matrix
# See here: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html
    


In [17]:
# The resulting matrix is in sparse format, we can transform it into dense
# Code prepared for you so you can see what results look like
text_tfidf = pd.DataFrame(text_tfidf.toarray())

In [18]:
# This is an example result, the matrix will contain lots of zero values, that is expected
# Some values will be non-zero
text_tfidf.head()

,0,1,2,3,4,5,6,7,8,9,...,16242,16243,16244,16245,16246,16247,16248,16249,16250,16251
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Now the Data is Ready for Classifier Usage

### Split Data into Train and Test sets (4 marks)

In [19]:
# Train/Test split


In [20]:
# You might need to reset index in each dataframe (depends on you how you do things)
# done for you to make it clearer
X_train.reset_index(inplace=True, drop=True)
X_test.reset_index(inplace=True, drop=True)
y_train.reset_index(inplace=True, drop=True)
y_test.reset_index(inplace=True, drop=True)

In [21]:
# You might need to take classes as separate columns (depends on you how you do things)
class1 = y_train['Level_1'].astype(str)
class2 = y_train['Level_2'].astype(str)
class3 = y_train['Level_3'].astype(str)

## Model training for the three levels (8 marks)

In [6]:
# Create and save model for level 1


In [7]:
## Create and save models for level 2


In [8]:
## Create and save models for level 3


## Predict the test set (8 marks)

In [ ]:
# Creating an empty Dataframe with column names only (depends on you how you do things)
results = pd.DataFrame(columns=['Level1_Pred', 'Level2_Pred', 'Level3_Pred'])

## Here we reload the saved models and use them to predict the levels
# load model for level 1 (done for you)
with open('level1.pk', 'rb') as nb:
    model = pickle.load(nb)

## loop through the test data, predict level 1, then based on that predict level 2
## and based on level 2 predict level 3 (you need to load saved models accordingly)

    

In [27]:
## After you add the predictions to the results dataframe
## they should look like this
results

,Level1_Pred,Level2_Pred,Level3_Pred
0,2CEC27F1,BAE8A,2ABA
1,2CEC27F1,BAE8A,2ABA
2,09BF5150,C7E19,D06E
3,4513C920,F4055,7C00
4,4513C920,F4055,7C00
...,...,...,...
2121,09BF5150,262E7,29B3
2122,69286F45,2D5A3,28A7
2123,3E1E0D78,9D9EE,05A0
2124,57164AC1,7B638,0F8B


## Compute Accuracy on each level (4 marks)
Now you have the predictions for each level (in the test data), and you also have the actual levels, you can compute the accurcay

In [10]:
# Level 1 accuracy


In [11]:
# Level 2 accuracy


In [12]:
# Level 3 accuracy


## Well done!